In [0]:
import scandir
import os
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import rpy2.robjects as ro
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import scipy as sp
import dill
import random
import cyvcf
from hdfstorehelper import HDFStoreHelper
import statsmodels.api as sm
import statsmodels.formula.api as smf
import operator
import traceback
%load_ext rpy2.ipython
r = ro.r

In [0]:
analysis_dir_imputed = "/home/cfriedline/eckertlab/gypsy_indiv/masked/analysis/samtools1.2_no_otis/beagle40/"
snp_file_gz_imputed = "isect.vcf.gz.sorted.gz"

analysis_dir_notimputed = "/home/cfriedline/eckertlab/gypsy_indiv/masked/analysis/samtools1.2_no_otis/notimputed/"
snp_file_gz_notimputed = "isect.vcf.gz.sorted.gz"

hdf_notimputed = HDFStoreHelper(os.path.join(analysis_dir_notimputed, "isect.hd5"))
hdf_all_notimputed = HDFStoreHelper(os.path.join(analysis_dir_notimputed, "gypsy_samtools12.vcf.gz.hd5"))
hdf_imputed = HDFStoreHelper(os.path.join(analysis_dir_imputed, "isect.hd5"))
hdf_all_imputed = HDFStoreHelper(os.path.join(analysis_dir_imputed, "gypsy_samtools12_imputed40.vcf.gz.hd5"))

In [0]:
hdf_imputed.get_group_names()

In [0]:
hdf_notimputed.get_group_names() == hdf_imputed.get_group_names()

In [0]:
sig_mass_notimp = hdf_notimputed.get('sig_snps_mass')
sig_mass_imp = hdf_imputed.get('sig_snps_mass')

sig_tdt_notimp = hdf_notimputed.get('sig_snps_tdt')
sig_tdt_imp = hdf_imputed.get('sig_snps_tdt')

sig_pd_notimp = hdf_notimputed.get('sig_snps_pd')
sig_pd_imp = hdf_imputed.get('sig_snps_pd')

In [0]:
relaxed_sig_mass_notimp = hdf_notimputed.get('relaxed_sig_snps_mass')
relaxed_sig_mass_imp = hdf_imputed.get('relaxed_sig_snps_mass')

relaxed_sig_tdt_notimp = hdf_notimputed.get('relaxed_sig_snps_tdt')
relaxed_sig_tdt_imp = hdf_imputed.get('relaxed_sig_snps_tdt')

relaxed_sig_pd_notimp = hdf_notimputed.get('relaxed_sig_snps_pd')
relaxed_sig_pd_imp = hdf_imputed.get('relaxed_sig_snps_pd')

In [0]:
sig_mass_imp

In [0]:
sig_mass_isect = set(sig_mass_notimp.rs.values).intersection(set(sig_mass_imp.rs.values))
sig_tdt_isect = set(sig_tdt_notimp.rs.values).intersection(set(sig_tdt_imp.rs.values))
sig_pd_isect = set(sig_pd_notimp.rs.values).intersection(set(sig_pd_imp.rs.values))

relaxed_sig_mass_isect = set(relaxed_sig_mass_notimp.rs.values).intersection(set(relaxed_sig_mass_imp.rs.values))
relaxed_sig_tdt_isect = set(relaxed_sig_tdt_notimp.rs.values).intersection(set(relaxed_sig_tdt_imp.rs.values))
relaxed_sig_pd_isect = set(relaxed_sig_pd_notimp.rs.values).intersection(set(relaxed_sig_pd_imp.rs.values))

In [0]:
[len(x) for x in [sig_mass_isect, 
                 sig_tdt_isect,
                 sig_pd_isect]]

In [0]:
[len(x) for x in [relaxed_sig_mass_isect, 
                 relaxed_sig_tdt_isect,
                 relaxed_sig_pd_isect]]

In [0]:
test = list(relaxed_sig_mass_isect)[0]
print test

In [0]:
relaxed_sig_mass_imp.ix[relaxed_sig_mass_imp.rs == test]

In [0]:
relaxed_sig_mass_notimp.ix[relaxed_sig_mass_notimp.rs == test]

In [0]:
hdf_imputed.get('mcmc_pd_hmean').shape

In [0]:
plt.scatter(hdf_imputed.get('mcmc_mass_hmean').postrb_hmean,hdf_notimputed.get('mcmc_mass_hmean').postrb_hmean)
plt.xlabel("imputed")
plt.ylabel("notimputed")
plt.show()

plt.scatter(hdf_imputed.get('mcmc_tdt_hmean').postrb_hmean,hdf_notimputed.get('mcmc_tdt_hmean').postrb_hmean)
plt.xlabel("imputed")
plt.ylabel("notimputed")
plt.show()

plt.scatter(hdf_imputed.get('mcmc_pd_hmean').postrb_hmean,hdf_notimputed.get('mcmc_pd_hmean').postrb_hmean)
plt.xlabel("imputed")
plt.ylabel("notimputed")
plt.show()

In [0]:
plt.scatter(hdf_imputed.get('mcmc_mass_hmean').betarb_hmean,hdf_notimputed.get('mcmc_mass_hmean').betarb_hmean)
plt.xlabel("imputed")
plt.ylabel("notimputed")
plt.show()

plt.scatter(hdf_imputed.get('mcmc_tdt_hmean').betarb_hmean,hdf_notimputed.get('mcmc_tdt_hmean').betarb_hmean)
plt.xlabel("imputed")
plt.ylabel("notimputed")
plt.show()

plt.scatter(hdf_imputed.get('mcmc_pd_hmean').betarb_hmean,hdf_notimputed.get('mcmc_pd_hmean').betarb_hmean)
plt.xlabel("imputed")
plt.ylabel("notimputed")
plt.show()

In [0]:
pimass_gt_imp = hdf_imputed.get('pimass_gt')
pimass_gt_notimp = hdf_notimputed.get('pimass_gt')
pimass_gt_notimp = pimass_gt_notimp.replace("NA", np.nan)

In [0]:
pimass_gt_imp.ix[:,2:] = pimass_gt_imp.ix[:,2:].astype(float)
pimass_gt_notimp.ix[:,2:] = pimass_gt_notimp.ix[:,2:].astype(float)

In [0]:
coefs = []
for i in xrange(len(pimass_gt_imp)):
    df = pd.DataFrame([pimass_gt_imp.ix[i,2:],pimass_gt_notimp.ix[i,2:]]).dropna(axis=1)
    coefs.append(np.corrcoef(df.ix[0,:], df.ix[1,:])[0,1])
    if i % 1000 == 0:
        print i

In [0]:
plt.hist(coefs)
plt.show()

In [0]:
total = 0
for i, elem in enumerate(coefs):
    if elem < 0:
        total += 1
        print i, elem
print total